# ES UAT BAN Prep for Migration Testing

This notebook prepares ES UAT BANs for migration testing by marking them with `BBF_Ban__c = True`.

## Purpose
In production, the `es_to_bbf_ban_creation_v12.ipynb` creates new -BBF BANs in OSS and marks them for migration.
Since UAT doesn't have OSS access, this notebook mimics that process by:
1. Identifying qualifying Orders (Active + NOT PA MARKET DECOM)
2. Finding their parent BANs
3. **EXCLUDING BANs with active PA MARKET DECOM orders** (cancelled/disconnected PA DECOM is OK)
4. Marking only eligible BANs with `BBF_Ban__c = True`
5. Extracting related Account and Address IDs for downstream migrations

## Central Policy (Same as Production)
**Include Orders where:**
- Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
- AND Project_Group__c NOT LIKE '%PA MARKET DECOM%'

**Exclude BANs where:**
- The BAN has ANY **active** PA MARKET DECOM order (Activated, Suspended, Disconnect in Progress)
- Cancelled/Disconnected PA DECOM orders do NOT exclude a BAN

## What's OK on a BAN
- ✅ Cancelled orders (any project group)
- ✅ Disconnected orders (any project group)
- ✅ PA MARKET DECOM orders that are cancelled/disconnected
- ✅ At least one active qualifying order

## Output
- Updates ES UAT `Billing_Invoice__c.BBF_Ban__c = True` for eligible BANs
- Excel file with:
  - BANs marked for migration
  - BANs excluded (with active PA DECOM) and why
  - Account IDs to migrate
  - Address IDs to migrate
  - Order counts and summary

## Safety
- `DRY_RUN = True` by default (shows what would be updated)
- `BAN_LIMIT` controls how many BANs to mark (0 = no limit)
- `MAX_ORDERS_PER_BAN` limits BANs to those with few orders (0 = no limit)
- Only marks BANs, doesn't create or delete anything

In [11]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from collections import defaultdict
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("Setup complete")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
Setup complete


In [12]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# === MIGRATION POLICY (same as production) ===
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]
WD_PROJECT_GROUP = "PA MARKET DECOM"  # Orders with this are EXCLUDED

# === RUN OPTIONS ===
DRY_RUN = False  # Set to False to actually update BANs
BAN_LIMIT = 0  # Max BANs to mark (0 = no limit) - SET TO 0 FOR FULL RUN
MAX_ORDERS_PER_BAN = 0  # Max orders per BAN (0 = no limit) - SET TO 0 FOR FULL RUN
EXCLUDE_MIXED_BANS = True  # If True, exclude BANs that have ANY non-migratable orders

# === OUTPUT ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_type = "dry_run" if DRY_RUN else "live_run"
output_file = f"uat_ban_prep_{run_type}_{timestamp}.xlsx"

print("=" * 70)
print("CONFIGURATION")
print("=" * 70)
print(f"DRY_RUN: {DRY_RUN}")
print(f"BAN_LIMIT: {BAN_LIMIT if BAN_LIMIT > 0 else 'No limit'}")
print(
    f"MAX_ORDERS_PER_BAN: {MAX_ORDERS_PER_BAN if MAX_ORDERS_PER_BAN > 0 else 'No limit'}"
)
print(f"EXCLUDE_MIXED_BANS: {EXCLUDE_MIXED_BANS}")
print(f"Output: {output_file}")
print(f"\nMigration Policy:")
print(f"  Active Statuses: {ACTIVE_STATUSES}")
print(f"  Exclude Project Group: {WD_PROJECT_GROUP}")
if EXCLUDE_MIXED_BANS:
    print(f"  Mixed BAN Handling: EXCLUDE (only mark 'pure' BANs)")

if not DRY_RUN:
    print("\n" + "!" * 70)
    print("WARNING: LIVE RUN - BANs will be updated with BBF_Ban__c = True")
    print("!" * 70)

CONFIGURATION
DRY_RUN: False
BAN_LIMIT: No limit
MAX_ORDERS_PER_BAN: No limit
EXCLUDE_MIXED_BANS: True
Output: uat_ban_prep_live_run_20260123_092517.xlsx

Migration Policy:
  Active Statuses: ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
  Exclude Project Group: PA MARKET DECOM
  Mixed BAN Handling: EXCLUDE (only mark 'pure' BANs)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [13]:
# === CONNECT TO ES UAT ===

print("=" * 70)
print("CONNECTING TO ES UAT")
print("=" * 70)

print("\nConnecting to ES UAT Salesforce...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"Connected to ES UAT: {es_sf.sf_instance}")

CONNECTING TO ES UAT

Connecting to ES UAT Salesforce...
Connected to ES UAT: everstream--uat.sandbox.my.salesforce.com


In [14]:
# === STEP 1: QUERY QUALIFYING ORDERS ===
# Orders that meet the migration criteria:
# - Status IN (Active statuses)
# - EXCLUDE: Project_Group__c LIKE '%PA MARKET DECOM%'
# All other Project_Group__c values (including NULL) are included.

print("\n" + "=" * 70)
print("STEP 1: QUERY QUALIFYING ORDERS")
print("=" * 70)

status_str = "','".join(ACTIVE_STATUSES)

# Note: The (Project_Group__c = null OR NOT LIKE) pattern is required because
# SOQL NULL comparisons don't work with NOT LIKE alone
orders_query = f"""
    SELECT Id, OrderNumber, Name, Service_ID__c,
           Status, Project_Group__c,
           Billing_Invoice__c, Billing_Invoice__r.Name, Billing_Invoice__r.Account__c,
           Billing_Invoice__r.Account__r.Name, Billing_Invoice__r.BBF_Ban__c,
           AccountId, Account.Name,
           Address_A__c, Address_A__r.Name,
           Address_Z__c, Address_Z__r.Name
    FROM Order
    WHERE Status IN ('{status_str}')
      AND (Project_Group__c = null OR (NOT Project_Group__c LIKE '%{WD_PROJECT_GROUP}%'))
      AND Billing_Invoice__c != null
    ORDER BY Billing_Invoice__r.Name
"""

print(f"Criteria:")
print(f"  Include: Status IN {ACTIVE_STATUSES}")
print(f"  Exclude: Project_Group__c LIKE '%{WD_PROJECT_GROUP}%'")
print(f"\nExecuting query...")

result = es_sf.query_all(orders_query)
qualifying_orders = result["records"]

print(f"\nFound {len(qualifying_orders):,} qualifying Orders")

if len(qualifying_orders) > 0:
    sample = qualifying_orders[0]
    print(f"\nSample Order:")
    print(f"  Order Number: {sample.get('OrderNumber', 'N/A')}")
    print(f"  Service ID:   {sample.get('Service_ID__c', 'N/A')}")
    print(f"  Status:       {sample.get('Status', 'N/A')}")
    print(f"  Project Group: {sample.get('Project_Group__c', '(none)')}")
    print(f"  BAN:          {sample.get('Billing_Invoice__r', {}).get('Name', 'N/A')}")
    print(f"  Account:      {sample.get('Account', {}).get('Name', 'N/A')}")


STEP 1: QUERY QUALIFYING ORDERS
Criteria:
  Include: Status IN ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
  Exclude: Project_Group__c LIKE '%PA MARKET DECOM%'

Executing query...

Found 13,535 qualifying Orders

Sample Order:
  Order Number: SOF-78413
  Service ID:   EV-DFBR-78413
  Status:       Activated
  Project Group: None
  BAN:          00-BUCK
  Account:      Buckeye Telesystem


In [15]:
# === STEP 1.5: IDENTIFY BANs WITH ACTIVE PA MARKET DECOM ORDERS ===
# A BAN should be excluded if it has ANY order that is:
#   - ACTIVE status (Activated, Suspended, Disconnect in Progress)
#   - AND Project_Group__c LIKE '%PA MARKET DECOM%'
#
# It's OK for a BAN to have:
#   - Cancelled/Disconnected orders (any project group)
#   - PA MARKET DECOM orders that are NOT active (e.g., cancelled)
#   - Draft orders, etc.
#
# The "poison" is specifically: Active PA MARKET DECOM orders

print("\n" + "=" * 70)
print("STEP 1.5: IDENTIFY BANs WITH ACTIVE PA MARKET DECOM ORDERS")
print("=" * 70)

if EXCLUDE_MIXED_BANS:
    # Query for orders that are BOTH active AND PA MARKET DECOM
    # These are the "poison" orders that disqualify a BAN
    status_str = "','".join(ACTIVE_STATUSES)

    poison_orders_query = f"""
        SELECT Billing_Invoice__c, Billing_Invoice__r.Name, 
               Status, Project_Group__c, OrderNumber
        FROM Order
        WHERE Billing_Invoice__c != null
          AND Status IN ('{status_str}')
          AND Project_Group__c LIKE '%{WD_PROJECT_GROUP}%'
    """

    print(f"\nQuerying for 'poison' Orders (Active PA MARKET DECOM)...")
    print(f"  Poison = Status IN {ACTIVE_STATUSES}")
    print(f"           AND Project_Group__c LIKE '%{WD_PROJECT_GROUP}%'")
    print(
        f"\nNote: Cancelled/Disconnected PA DECOM orders are OK - only ACTIVE PA DECOM is excluded"
    )

    poison_result = es_sf.query_all(poison_orders_query)
    poison_orders = poison_result["records"]

    print(f"\nFound {len(poison_orders):,} Active PA MARKET DECOM Orders")

    # Build set of "tainted" BAN IDs and track reasons
    mixed_ban_ids = set()
    mixed_ban_details = (
        {}
    )  # BAN ID -> {ban_name, reasons: [{order, status, project_group}]}

    for order in poison_orders:
        ban_id = order.get("Billing_Invoice__c")
        if ban_id:
            mixed_ban_ids.add(ban_id)

            if ban_id not in mixed_ban_details:
                ban_info = order.get("Billing_Invoice__r", {}) or {}
                mixed_ban_details[ban_id] = {
                    "ban_id": ban_id,
                    "ban_name": ban_info.get("Name", "N/A"),
                    "poison_orders": [],
                    "poison_count": 0,
                }

            mixed_ban_details[ban_id]["poison_orders"].append(
                {
                    "order_number": order.get("OrderNumber", "N/A"),
                    "status": order.get("Status", "N/A"),
                    "project_group": order.get("Project_Group__c", "(none)"),
                }
            )
            mixed_ban_details[ban_id]["poison_count"] += 1

    print(
        f"\nBANs with Active PA MARKET DECOM (will be excluded): {len(mixed_ban_ids):,}"
    )

    # Show sample of excluded BANs
    if mixed_ban_details:
        print(f"\nSample excluded BANs (first 5):")
        for i, (ban_id, details) in enumerate(list(mixed_ban_details.items())[:5], 1):
            sample_order = details["poison_orders"][0]
            print(
                f"  {i}. {details['ban_name']} - {details['poison_count']} active PA DECOM orders"
            )
            print(
                f"      Example: {sample_order['order_number']} - Status: {sample_order['status']}"
            )
else:
    mixed_ban_ids = set()
    mixed_ban_details = {}
    print("\nPA MARKET DECOM exclusion is DISABLED (EXCLUDE_MIXED_BANS = False)")
    print("BANs with active PA DECOM orders will be included.")


STEP 1.5: IDENTIFY BANs WITH ACTIVE PA MARKET DECOM ORDERS

Querying for 'poison' Orders (Active PA MARKET DECOM)...
  Poison = Status IN ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
           AND Project_Group__c LIKE '%PA MARKET DECOM%'

Note: Cancelled/Disconnected PA DECOM orders are OK - only ACTIVE PA DECOM is excluded

Found 819 Active PA MARKET DECOM Orders

BANs with Active PA MARKET DECOM (will be excluded): 31

Sample excluded BANs (first 5):
  1. E91910226264-W - 126 active PA DECOM orders
      Example: SOF-28725 - Status: Activated
  2. P91910228765-R - 15 active PA DECOM orders
      Example: SOF-34766 - Status: Activated
  3. P91910228344-R - 6 active PA DECOM orders
      Example: SOF-34771 - Status: Activated
  4. P91910228385-R - 1 active PA DECOM orders
      Example: SOF-34836 - Status: Activated
  5. P91910228369-R - 286 active PA DECOM orders
      Example: SOF-34839 - Status: Disconnect in Progress


In [16]:
# === STEP 2: EXTRACT UNIQUE IDs ===
# Get unique BANs, Accounts, and Addresses from qualifying Orders
# IMPORTANT: Exclude BANs that have ANY active PA MARKET DECOM orders

print("\n" + "=" * 70)
print("STEP 2: EXTRACT UNIQUE IDs (excluding BANs with active PA DECOM)")
print("=" * 70)

# Collect unique IDs
ban_data = (
    {}
)  # BAN SFID -> {name, account_id, account_name, order_count, already_marked}
account_ids = set()
address_ids = set()
skipped_order_count = 0

for order in qualifying_orders:
    # BAN
    ban_id = order.get("Billing_Invoice__c")
    if ban_id:
        # Skip if this BAN has active PA DECOM orders
        if EXCLUDE_MIXED_BANS and ban_id in mixed_ban_ids:
            skipped_order_count += 1
            continue

        if ban_id not in ban_data:
            ban_info = order.get("Billing_Invoice__r", {}) or {}
            account_info = ban_info.get("Account__r", {}) or {}
            ban_data[ban_id] = {
                "ban_id": ban_id,
                "ban_name": ban_info.get("Name", "N/A"),
                "account_id": ban_info.get("Account__c"),
                "account_name": account_info.get("Name", "N/A"),
                "already_marked": ban_info.get("BBF_Ban__c", False),
                "order_count": 0,
                "orders": [],
            }
        ban_data[ban_id]["order_count"] += 1
        ban_data[ban_id]["orders"].append(order.get("OrderNumber", "N/A"))

        # Account (from BAN's Account, not Order's Account)
        if ban_info.get("Account__c"):
            account_ids.add(ban_info["Account__c"])

        # Addresses (only for eligible BANs)
        if order.get("Address_A__c"):
            address_ids.add(order["Address_A__c"])
        if order.get("Address_Z__c"):
            address_ids.add(order["Address_Z__c"])

# Convert to list and sort by order count
ban_list = sorted(ban_data.values(), key=lambda x: x["order_count"], reverse=True)

print(f"\nQualifying Orders processed: {len(qualifying_orders):,}")
if EXCLUDE_MIXED_BANS:
    print(f"Orders skipped (on BANs with active PA DECOM): {skipped_order_count:,}")
    print(f"Orders on eligible BANs: {len(qualifying_orders) - skipped_order_count:,}")

print(f"\nUnique counts from ELIGIBLE BANs:")
print(f"  Eligible BANs: {len(ban_list):,}")
print(f"  Accounts:      {len(account_ids):,}")
print(f"  Addresses:     {len(address_ids):,}")

if EXCLUDE_MIXED_BANS:
    print(f"\nPA DECOM Exclusion Summary:")
    print(f"  BANs excluded (have active PA DECOM): {len(mixed_ban_ids):,}")

# Check how many are already marked
already_marked = [b for b in ban_list if b["already_marked"]]
not_marked = [b for b in ban_list if not b["already_marked"]]

print(f"\nBAN Status (eligible BANs only):")
print(f"  Already BBF_Ban__c = True: {len(already_marked):,}")
print(f"  Not yet marked:            {len(not_marked):,}")

print(f"\nTop 10 eligible BANs by order count:")
for i, ban in enumerate(ban_list[:10], 1):
    marked = "[MARKED]" if ban["already_marked"] else ""
    print(
        f"  {i}. {ban['ban_name']} - {ban['order_count']} orders - {ban['account_name']} {marked}"
    )


STEP 2: EXTRACT UNIQUE IDs (excluding BANs with active PA DECOM)

Qualifying Orders processed: 13,535
Orders skipped (on BANs with active PA DECOM): 3,559
Orders on eligible BANs: 9,976

Unique counts from ELIGIBLE BANs:
  Eligible BANs: 2,499
  Accounts:      2,258
  Addresses:     8,165

PA DECOM Exclusion Summary:
  BANs excluded (have active PA DECOM): 31

BAN Status (eligible BANs only):
  Already BBF_Ban__c = True: 21
  Not yet marked:            2,478

Top 10 eligible BANs by order count:
  1. E91910205540-R - 551 orders - T-Mobile Small Cell 
  2. A91910221190-R - 326 orders - Meijer, Inc 
  3. A91910155661-W - 273 orders - CenturyLink - dba Lumen 
  4. E91910156370-R - 245 orders - Northeast Ohio Network for Educational Technology-NEONET 
  5. E91910126852-R - 216 orders - Snip Internet, LLC (Zentro) 
  6. R91910218410-R - 172 orders - Bedrock Management Services LLC 
  7. LYNX051602 - 170 orders - AT&T Corp 
  8. E91910158517-R - 166 orders - University Hospital Health Syste

In [17]:
# === STEP 3: SELECT BANs TO MARK ===
# Apply filters:
# 1. Already filtered out BANs with active PA DECOM in Step 2
# 2. Filter out already-marked BANs
# 3. Filter by MAX_ORDERS_PER_BAN (if set)
# 4. Apply BAN_LIMIT (if set)

print("\n" + "=" * 70)
print("STEP 3: SELECT BANs TO MARK")
print("=" * 70)

# Only mark BANs that aren't already marked
bans_to_mark = not_marked.copy()
print(f"\nStarting with {len(bans_to_mark):,} eligible BANs (not already marked)")

# Filter by max orders per BAN (optional - for testing)
if MAX_ORDERS_PER_BAN > 0:
    before_filter = len(bans_to_mark)
    bans_to_mark = [b for b in bans_to_mark if b["order_count"] <= MAX_ORDERS_PER_BAN]
    print(f"\nFiltering by MAX_ORDERS_PER_BAN <= {MAX_ORDERS_PER_BAN}:")
    print(f"  Before: {before_filter:,} BANs")
    print(f"  After:  {len(bans_to_mark):,} BANs")

# Sort by order count ascending (smallest first)
bans_to_mark = sorted(bans_to_mark, key=lambda x: x["order_count"])

# Apply BAN limit (optional - for testing)
if BAN_LIMIT > 0 and len(bans_to_mark) > BAN_LIMIT:
    print(f"\nApplying BAN_LIMIT: {BAN_LIMIT} BANs")
    bans_to_mark = bans_to_mark[:BAN_LIMIT]

print(f"\nBANs selected for marking: {len(bans_to_mark):,}")

# Recalculate related IDs based on selected BANs
selected_ban_ids = {b["ban_id"] for b in bans_to_mark}
selected_account_ids = {b["account_id"] for b in bans_to_mark if b["account_id"]}

# Get addresses only for orders on selected BANs
selected_address_ids = set()
selected_order_count = 0
for order in qualifying_orders:
    if order.get("Billing_Invoice__c") in selected_ban_ids:
        selected_order_count += 1
        if order.get("Address_A__c"):
            selected_address_ids.add(order["Address_A__c"])
        if order.get("Address_Z__c"):
            selected_address_ids.add(order["Address_Z__c"])

print(f"\nSelected migration scope:")
print(f"  BANs:      {len(bans_to_mark):,}")
print(f"  Accounts:  {len(selected_account_ids):,}")
print(f"  Addresses: {len(selected_address_ids):,}")
print(f"  Orders:    {selected_order_count:,}")

if len(bans_to_mark) > 0:
    print(f"\nBANs to mark (first 20):")
    for i, ban in enumerate(bans_to_mark[:20], 1):
        print(
            f"  {i}. {ban['ban_name']} ({ban['order_count']} orders) - {ban['account_name']}"
        )
    if len(bans_to_mark) > 20:
        print(f"  ... and {len(bans_to_mark) - 20} more")
else:
    print("\nNo BANs to mark (all eligible BANs already have BBF_Ban__c = True)")


STEP 3: SELECT BANs TO MARK

Starting with 2,478 eligible BANs (not already marked)

BANs selected for marking: 2,478

Selected migration scope:
  BANs:      2,478
  Accounts:  2,244
  Addresses: 8,145
  Orders:    9,954

BANs to mark (first 20):
  1. A91910103745-W (1 orders) - Springport Telephone Company
  2. A91910103778-W (1 orders) - Barry County
  3. A91910103786-W (1 orders) - Blanchard Telephone
  4. A91910103810-W (1 orders) - Bloomingdale
  5. A91910104453-R (1 orders) - L&W Engineering
  6. A91910105112-W (1 orders) - Darby Ready Mix Concrete
  7. A91910129815-W (1 orders) - Level 3 Communications LLC -dba Lumen
  8. A91910147148-R (1 orders) - Techcraft Seating Systems
  9. A91910150100-R (1 orders) - Ensign Equipment
  10. A91910150316-R (1 orders) - G.P. Reeves Inc.
  11. A91910150423-R (1 orders) - american 1 credit union
  12. A91910150480-W (1 orders) - T-Mobile
  13. A91910150605-R (1 orders) - Innocademy Allegan Campus
  14. A91910150720-R (1 orders) - Hospitality 

In [18]:
# === STEP 4: UPDATE BANs ===
# Set BBF_Ban__c = True on selected BANs

print("\n" + "=" * 70)
print("STEP 4: UPDATE BANs")
print("=" * 70)

update_results = []

if len(bans_to_mark) == 0:
    print("\nNo BANs to update.")
elif DRY_RUN:
    print(f"\nDRY RUN - Would update {len(bans_to_mark)} BANs")
    for ban in bans_to_mark:
        update_results.append(
            {
                "ban_id": ban["ban_id"],
                "ban_name": ban["ban_name"],
                "account_name": ban["account_name"],
                "order_count": ban["order_count"],
                "status": "WOULD_UPDATE",
                "error": None,
            }
        )
    print("\nNo changes made (DRY_RUN = True)")
else:
    print(f"\nUpdating {len(bans_to_mark)} BANs with BBF_Ban__c = True...")

    # Build update records
    updates = [{"Id": ban["ban_id"], "BBF_Ban__c": True} for ban in bans_to_mark]

    try:
        results = es_sf.bulk.Billing_Invoice__c.update(updates)

        success_count = 0
        error_count = 0

        for i, result in enumerate(results):
            ban = bans_to_mark[i]
            if result["success"]:
                success_count += 1
                update_results.append(
                    {
                        "ban_id": ban["ban_id"],
                        "ban_name": ban["ban_name"],
                        "account_name": ban["account_name"],
                        "order_count": ban["order_count"],
                        "status": "UPDATED",
                        "error": None,
                    }
                )
            else:
                error_count += 1
                update_results.append(
                    {
                        "ban_id": ban["ban_id"],
                        "ban_name": ban["ban_name"],
                        "account_name": ban["account_name"],
                        "order_count": ban["order_count"],
                        "status": "ERROR",
                        "error": str(result["errors"]),
                    }
                )

        print(f"\nUpdate Results:")
        print(f"  Successfully updated: {success_count}")
        print(f"  Errors: {error_count}")

        if error_count > 0:
            print(f"\nFirst 5 errors:")
            for r in update_results:
                if r["status"] == "ERROR":
                    print(f"  - {r['ban_name']}: {r['error']}")

    except Exception as e:
        print(f"\nError during bulk update: {e}")
        for ban in bans_to_mark:
            update_results.append(
                {
                    "ban_id": ban["ban_id"],
                    "ban_name": ban["ban_name"],
                    "account_name": ban["account_name"],
                    "order_count": ban["order_count"],
                    "status": "ERROR",
                    "error": str(e),
                }
            )


STEP 4: UPDATE BANs

Updating 2478 BANs with BBF_Ban__c = True...

Update Results:
  Successfully updated: 2477
  Errors: 1

First 5 errors:
  - E91910223055-R: [{'statusCode': 'FIELD_CUSTOM_VALIDATION_EXCEPTION', 'message': 'Billing Email is not valid', 'fields': ['Billing_E_mail__c']}]


In [19]:
# === STEP 5: CREATE EXCEL OUTPUT ===

print("\n" + "=" * 70)
print("STEP 5: CREATE EXCEL OUTPUT")
print("=" * 70)

wb = Workbook()

# Styling
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
header_fill_green = PatternFill("solid", fgColor="70AD47")
header_fill_orange = PatternFill("solid", fgColor="ED7D31")
header_fill_red = PatternFill("solid", fgColor="C00000")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)


def style_header(ws, headers, fill_color):
    for col, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = fill_color
        cell.alignment = header_alignment
        cell.border = thin_border


def auto_width(ws):
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        ws.column_dimensions[column].width = min(max_length + 2, 50)


status_colors = {
    "WOULD_UPDATE": "C6EFCE",
    "UPDATED": "C6EFCE",
    "ERROR": "FF6666",
    "ALREADY_MARKED": "FFEB9C",
}

# === Sheet 1: Summary ===
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["ES UAT BAN Prep Summary"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE RUN"])
ws1.append(["BAN Limit:", BAN_LIMIT if BAN_LIMIT > 0 else "No limit"])
ws1.append(
    [
        "Max Orders Per BAN:",
        MAX_ORDERS_PER_BAN if MAX_ORDERS_PER_BAN > 0 else "No limit",
    ]
)
ws1.append(["Exclude Active PA DECOM:", "Yes" if EXCLUDE_MIXED_BANS else "No"])
ws1.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append([])
ws1.append(["Migration Policy:"])
ws1[f"A{ws1.max_row}"].font = Font(bold=True)
ws1.append(["  Active Statuses:", ", ".join(ACTIVE_STATUSES)])
ws1.append(["  Exclude Project Group:", WD_PROJECT_GROUP])
ws1.append([])
ws1.append(["Qualifying Orders Found:", len(qualifying_orders)])
ws1.append([])
ws1.append(["Active PA MARKET DECOM Analysis:"])
ws1[f"A{ws1.max_row}"].font = Font(bold=True)
ws1.append(
    ["  Active PA DECOM Orders:", len(poison_orders) if EXCLUDE_MIXED_BANS else "N/A"]
)
ws1.append(
    [
        "  BANs Excluded (have active PA DECOM):",
        len(mixed_ban_ids) if EXCLUDE_MIXED_BANS else "N/A",
    ]
)
ws1.append([])
ws1.append(["Eligible BANs (no active PA DECOM):"])
ws1[f"A{ws1.max_row}"].font = Font(bold=True)
ws1.append(["  Total Eligible BANs:", len(ban_list)])
ws1.append(["  Already Marked:", len(already_marked)])
ws1.append(["  Not Yet Marked:", len(not_marked)])
ws1.append(["  Total Accounts:", len(account_ids)])
ws1.append(["  Total Addresses:", len(address_ids)])
ws1.append([])
ws1.append(["Selected for This Run:"])
ws1[f"A{ws1.max_row}"].font = Font(bold=True)
ws1.append(["  BANs to Mark:", len(bans_to_mark)])
ws1.append(["  Related Accounts:", len(selected_account_ids)])
ws1.append(["  Related Addresses:", len(selected_address_ids)])
ws1.append(["  Related Orders:", selected_order_count])
ws1.append([])
if update_results:
    ws1.append(["Update Results:"])
    ws1[f"A{ws1.max_row}"].font = Font(bold=True)
    status_counts = defaultdict(int)
    for r in update_results:
        status_counts[r["status"]] += 1
    for status, count in status_counts.items():
        ws1.append([f"  {status}:", count])

# === Sheet 2: BANs to Migrate ===
ws2 = wb.create_sheet("BANs to Migrate")
headers2 = [
    "BAN ID",
    "BAN Name",
    "Account ID",
    "Account Name",
    "Order Count",
    "Status",
    "Error",
]
ws2.append(headers2)
style_header(ws2, headers2, header_fill)

for r in update_results:
    ban = next((b for b in bans_to_mark if b["ban_id"] == r["ban_id"]), {})
    ws2.append(
        [
            r["ban_id"],
            r["ban_name"],
            ban.get("account_id", ""),
            r["account_name"],
            r["order_count"],
            r["status"],
            r["error"] or "",
        ]
    )
    row_idx = ws2.max_row
    fill_color = status_colors.get(r["status"], "FFFFFF")
    for col in range(1, len(headers2) + 1):
        ws2.cell(row=row_idx, column=col).fill = PatternFill(
            "solid", fgColor=fill_color
        )

ws2.freeze_panes = "A2"
auto_width(ws2)

# === Sheet 3: BANs Excluded (Active PA DECOM) ===
ws3 = wb.create_sheet("BANs Excluded-PA DECOM")
headers3 = [
    "BAN ID",
    "BAN Name",
    "Active PA DECOM Orders",
    "Sample Order",
    "Sample Status",
    "Sample Project Group",
]
ws3.append(headers3)
style_header(ws3, headers3, header_fill_red)

if EXCLUDE_MIXED_BANS and mixed_ban_details:
    for ban_id, details in sorted(
        mixed_ban_details.items(), key=lambda x: x[1]["poison_count"], reverse=True
    ):
        sample = details["poison_orders"][0] if details["poison_orders"] else {}
        ws3.append(
            [
                ban_id,
                details["ban_name"],
                details["poison_count"],
                sample.get("order_number", ""),
                sample.get("status", ""),
                sample.get("project_group", ""),
            ]
        )

ws3.freeze_panes = "A2"
auto_width(ws3)

# === Sheet 4: Already Marked BANs ===
ws4 = wb.create_sheet("Already Marked")
headers4 = ["BAN ID", "BAN Name", "Account ID", "Account Name", "Order Count"]
ws4.append(headers4)
style_header(ws4, headers4, header_fill_orange)

for ban in already_marked:
    ws4.append(
        [
            ban["ban_id"],
            ban["ban_name"],
            ban["account_id"],
            ban["account_name"],
            ban["order_count"],
        ]
    )

ws4.freeze_panes = "A2"
auto_width(ws4)

# === Sheet 5: Account IDs ===
ws5 = wb.create_sheet("Account IDs")
headers5 = ["Account ID"]
ws5.append(headers5)
style_header(ws5, headers5, header_fill_green)

for acc_id in sorted(selected_account_ids):
    ws5.append([acc_id])

ws5.freeze_panes = "A2"
auto_width(ws5)

# === Sheet 6: Address IDs ===
ws6 = wb.create_sheet("Address IDs")
headers6 = ["Address ID"]
ws6.append(headers6)
style_header(ws6, headers6, header_fill_green)

for addr_id in sorted(selected_address_ids):
    ws6.append([addr_id])

ws6.freeze_panes = "A2"
auto_width(ws6)

# === Sheet 7: All Qualifying Orders (for selected BANs) ===
ws7 = wb.create_sheet("Qualifying Orders")
headers7 = [
    "Order Number",
    "Service ID",
    "Status",
    "BAN Name",
    "Account Name",
    "Address A",
    "Address Z",
]
ws7.append(headers7)
style_header(ws7, headers7, header_fill)

for order in qualifying_orders:
    # Only include orders for selected BANs
    if order.get("Billing_Invoice__c") in selected_ban_ids:
        ws7.append(
            [
                order.get("OrderNumber", ""),
                order.get("Service_ID__c", ""),
                order.get("Status", ""),
                (
                    order.get("Billing_Invoice__r", {}).get("Name", "")
                    if order.get("Billing_Invoice__r")
                    else ""
                ),
                (
                    order.get("Account", {}).get("Name", "")
                    if order.get("Account")
                    else ""
                ),
                order.get("Address_A__c", ""),
                order.get("Address_Z__c", ""),
            ]
        )

ws7.freeze_panes = "A2"
auto_width(ws7)

# Save
wb.save(output_file)
print(f"\nExcel output saved to: {output_file}")
print(f"  Sheet 1: Summary")
print(f"  Sheet 2: BANs to Migrate ({len(update_results)} BANs)")
print(
    f"  Sheet 3: BANs Excluded-PA DECOM ({len(mixed_ban_details) if EXCLUDE_MIXED_BANS else 0} BANs)"
)
print(f"  Sheet 4: Already Marked ({len(already_marked)} BANs)")
print(f"  Sheet 5: Account IDs ({len(selected_account_ids)} accounts)")
print(f"  Sheet 6: Address IDs ({len(selected_address_ids)} addresses)")
print(f"  Sheet 7: Qualifying Orders ({selected_order_count} orders)")


STEP 5: CREATE EXCEL OUTPUT

Excel output saved to: uat_ban_prep_live_run_20260123_092517.xlsx
  Sheet 1: Summary
  Sheet 2: BANs to Migrate (2478 BANs)
  Sheet 3: BANs Excluded-PA DECOM (31 BANs)
  Sheet 4: Already Marked (21 BANs)
  Sheet 5: Account IDs (2244 accounts)
  Sheet 6: Address IDs (8145 addresses)
  Sheet 7: Qualifying Orders (9954 orders)


In [20]:
# === FINAL SUMMARY ===

print("\n" + "=" * 70)
print("UAT BAN PREP COMPLETE")
print("=" * 70)

print(f"\nRun Type: {'DRY RUN' if DRY_RUN else 'LIVE RUN'}")

if EXCLUDE_MIXED_BANS:
    print(f"\nActive PA MARKET DECOM Exclusion: ENABLED")
    print(f"  Active PA DECOM orders found: {len(poison_orders):,}")
    print(f"  BANs excluded (have active PA DECOM): {len(mixed_ban_ids):,}")
    print(f"  (Cancelled/Disconnected PA DECOM orders are OK)")

print(f"\nMigration Scope:")
print(f"  Eligible BANs: {len(bans_to_mark):,}")
print(f"  Accounts:      {len(selected_account_ids):,}")
print(f"  Addresses:     {len(selected_address_ids):,}")
print(f"  Orders:        {selected_order_count:,}")

print(f"\nOutput file: {output_file}")

if DRY_RUN:
    print("\n" + "-" * 70)
    print("DRY RUN complete. No BANs were updated.")
    print("To mark BANs for migration, set DRY_RUN = False and re-run.")
    print("-" * 70)
else:
    print("\n" + "-" * 70)
    print("LIVE RUN complete. BANs have been marked with BBF_Ban__c = True.")
    print("\nNext Steps:")
    print("  1. Run 01_location_migration.ipynb")
    print("  2. Run 02_account_migration.ipynb")
    print("  3. Run 03_contact_migration.ipynb")
    print("  4. Run 04_ban_migration.ipynb")
    print("  5. Run 05_service_migration.ipynb")
    print("  6. Run 06_service_charge_migration.ipynb")
    print("  7. Run 07_offnet_migration.ipynb")
    print("-" * 70)


UAT BAN PREP COMPLETE

Run Type: LIVE RUN

Active PA MARKET DECOM Exclusion: ENABLED
  Active PA DECOM orders found: 819
  BANs excluded (have active PA DECOM): 31
  (Cancelled/Disconnected PA DECOM orders are OK)

Migration Scope:
  Eligible BANs: 2,478
  Accounts:      2,244
  Addresses:     8,145
  Orders:        9,954

Output file: uat_ban_prep_live_run_20260123_092517.xlsx

----------------------------------------------------------------------
LIVE RUN complete. BANs have been marked with BBF_Ban__c = True.

Next Steps:
  1. Run 01_location_migration.ipynb
  2. Run 02_account_migration.ipynb
  3. Run 03_contact_migration.ipynb
  4. Run 04_ban_migration.ipynb
  5. Run 05_service_migration.ipynb
  6. Run 06_service_charge_migration.ipynb
  7. Run 07_offnet_migration.ipynb
----------------------------------------------------------------------


---
## Configuration Options

| Option | Default | Description |
|--------|---------|-------------|
| `DRY_RUN` | True | If True, shows what would be updated without making changes |
| `BAN_LIMIT` | 0 | Max BANs to mark (0 = no limit) |
| `MAX_ORDERS_PER_BAN` | 0 | Max orders per BAN (0 = no limit) |
| `EXCLUDE_MIXED_BANS` | True | If True, exclude BANs that have any ACTIVE PA MARKET DECOM orders |

## BAN Exclusion Logic

When `EXCLUDE_MIXED_BANS = True`:

**A BAN is EXCLUDED if it has ANY order that is:**
- Status IN (Activated, Suspended, Disconnect in Progress) **AND**
- Project_Group__c LIKE '%PA MARKET DECOM%'

**A BAN is OK to migrate if it has:**
- ✅ Cancelled orders (any project group)
- ✅ Disconnected orders (any project group)  
- ✅ Draft orders (any project group)
- ✅ PA MARKET DECOM orders that are NOT active (cancelled, disconnected)
- ✅ At least one active qualifying order (Active status + NOT PA DECOM)

**Examples:**

| BAN Has | Result |
|---------|--------|
| 5 Activated orders, 0 PA DECOM | ✅ MARKED |
| 5 Activated orders + 1 Cancelled PA DECOM | ✅ MARKED (cancelled PA DECOM is OK) |
| 5 Activated orders + 1 Active PA DECOM | ❌ EXCLUDED |
| 3 Cancelled orders + 1 Activated PA DECOM | ❌ EXCLUDED (has active PA DECOM) |
| Only Cancelled/Disconnected orders | Not in qualifying set (no active orders) |

## Next Steps

After running this notebook with `DRY_RUN = False`:

The downstream migration notebooks will automatically filter based on `BBF_Ban__c = true`:

```python
# Account migration - only migrate Accounts linked to BBF_Ban__c = true BANs
WHERE Id IN (
    SELECT Account__c FROM Billing_Invoice__c WHERE BBF_Ban__c = true
)

# Location migration - only migrate Addresses from qualifying Orders
WHERE Id IN (
    SELECT Address_A__c FROM Order
    WHERE Billing_Invoice__r.BBF_Ban__c = true
    UNION
    SELECT Address_Z__c FROM Order
    WHERE Billing_Invoice__r.BBF_Ban__c = true
)
```

## Reset for Re-Testing

To reset and re-run:

### Clear BBF_Ban__c in ES UAT
```apex
List<Billing_Invoice__c> bans = [SELECT Id, BBF_Ban__c FROM Billing_Invoice__c WHERE BBF_Ban__c = true];
System.debug('Found ' + bans.size() + ' BANs to reset');
for (Billing_Invoice__c bi : bans) {
    bi.BBF_Ban__c = false;
}
update bans;
```

### Clear BBF_New_Id__c from migrated objects
```apex
// Accounts
List<Account> accounts = [SELECT Id FROM Account WHERE BBF_New_Id__c != null];
for (Account a : accounts) { a.BBF_New_Id__c = null; }
update accounts;

// Contacts
List<Contact> contacts = [SELECT Id FROM Contact WHERE BBF_New_Id__c != null];
for (Contact c : contacts) { c.BBF_New_Id__c = null; }
update contacts;

// Addresses
List<Address__c> addresses = [SELECT Id FROM Address__c WHERE BBF_New_Id__c != null];
for (Address__c a : addresses) { a.BBF_New_Id__c = null; }
update addresses;

// BANs
List<Billing_Invoice__c> bans = [SELECT Id FROM Billing_Invoice__c WHERE BBF_New_Id__c != null];
for (Billing_Invoice__c bi : bans) { bi.BBF_New_Id__c = null; }
update bans;

// Orders
List<Order> orders = [SELECT Id FROM Order WHERE BBF_New_Id__c != null];
for (Order o : orders) { o.BBF_New_Id__c = null; }
update orders;
```